# Community Area Population from a pdf File

by Luc Anselin (anselin@uchicago.edu) (8/22/2016)

Pulling the data for the Chicago Community Area 2010 population from a pdf file, available
at http://www.cityofchicago.org/city/en/depts/dcd/supp_info/community_area_2000and2010censuspopulationcomparisons.html.
This link is to a pdf file that contains a table with the neighborhood ID, the neighborhood name, the populations
for 2010 and 2000, the difference between the two years and the percentage difference.


Note: this is written with R beginners in mind, more seasoned R users can probably skip most of the comments.

For more extensive details about each function, see the R (or RStudio) help files.

Packages used:

- **pdftools**

### Extracting the content from a pdf file

A pdf file is difficult to handle as a source of data, since it doesn't contain tags like an html file.
We will use the **pdftools** package that allows us to turn the contents of a pdf file into a list of long character strings,
one for each page of the document. This packages is not installed by default, so you may have to use
**install.packages("pdftools")** if it is not installed.

The resulting data structure is somewhat complex and not necessarily easy to parse. However, in our case, the table has such a simple structure that we can extract the population values by doing some sleuthing on which columns
contain those values. This will illustrate the power of the various parsing and text extraction functions available in R.

We start by turning the pdf into a list of text strings, and then organize that list so that it only contains the table entries for the 77 community area neighborhoods.



#### Reading the pdf file

We use the **pdf_text** function from the **pdftools** to turn the pdf file into a list of character strings, one
for each page.

In [1]:
library(pdftools)
dat <- pdf_text("Census_2010_and_2000_CA_Populations.pdf")

We check the contents of the **dat** object.

In [4]:
dat

[1] "                          CITY OF CHICAGO\n                        CENSUS 2010 AND 2000\n                                             Population\nNum      Community Area     2010        2,000      Difference Percentage\n 1  Rogers Park              54,991     63,484        -8,493    -13.4%\n 2  West Ridge               71,942     73,199        -1,257     -1.7%\n 3  Uptown                   56,362     63,551        -7,189    -11.3%\n 4  Lincoln Square           39,493     44,574        -5,081    -11.4%\n 5  North Center             31,867     31,895          -28      -0.1%\n 6  Lake View                94,368     94,817         -449      -0.5%\n 7  Lincoln Park             64,116     64,320         -204      -0.3%\n 8  Near North Side          80,484     72,811        7,673      10.5%\n 9  Edison Park              11,187     11,259          -72      -0.6%\n 10 Norwood Park             37,023     37,669         -646      -1.7%\n 11 Jefferson Park           25,448     25,859         -411      -1.6%\n 12 Forest Glen              18,508     18,165         343       1.9%\n 13 North Park               17,931     18,514         -583      -3.1%\n 14 Albany Park              51,542     57,655        -6,113    -10.6%\n 15 Portage Park             64,124     65,340        -1,216     -1.9%\n 16 Irving Park              53,359     58,643        -5,284     -9.0%\n 17 Dunning                  41,932     42,164         -232      -0.6%\n 18 Montclare                13,426     12,646         780       6.2%\n 19 Belmont Cragin           78,743     78,144         599       0.8%\n 20 Hermosa                  25,010     26,908        -1,898     -7.1%\n 21 Avondale                 39,262     43,083        -3,821     -8.9%\n 22 Logan Square             73,595     82,715        -9,120    -11.0%\n 23 Humboldt Park            56,323     65,836        -9,513    -14.4%\n 24 West Town                81,432     87,435        -6,003     -6.9%\n 25 Austin                   98,514    117,527       -19,013    -16.2%\n 26 West Garfield Park       18,001     23,019        -5,018    -21.8%\n 27 East Garfield Park       20,567     20,881         -314      -1.5%\n 28 Near West Side           54,881     46,419        8,462      18.2%\n 29 North Lawndale           35,912     41,768        -5,856    -14.0%\n 30 South Lawndale           79,288     91,071       -11,783    -12.9%\n 31 Lower West Side          35,769     44,031        -8,262    -18.8%\n 32 Loop                     29,283     16,388        12,895     78.7%\n 33 Near South Side          21,390     9,509         11,881    124.9%\n 34 Armour Square            13,391     12,032        1,359      11.3%\n 35 Douglas                  18,238     26,470        -8,232    -31.1%\n 36 Oakland                    5,918    6,110          -192      -3.1%\n 37 Fuller Park                2,876    3,420          -544     -15.9%\n 38 Grand Boulevard          21,929     28,006        -6,077    -21.7%\n 39 Kenwood                  17,841     18,363         -522      -2.8%\n 40 Washington Park          11,717     14,146        -2,429    -17.2%\n"                
[2] "                             CITY OF CHICAGO\n                           CENSUS 2010 AND 2000\n                                                 Population\nNum     Community Area         2010         2,000      Difference Percentage\n 41 Hyde Park                    25,681     29,920         -4,239   -14.2%\n 42 Woodlawn                     25,983     27,086         -1,103    -4.1%\n 43 South Shore                  49,767     61,556        -11,789   -19.2%\n 44 Chatham                      31,028     37,275         -6,247   -16.8%\n 45 Avalon Park                  10,185     11,147          -962     -8.6%\n 46 South Chicago                31,198     38,596         -7,398   -19.2%\n 47 Burnside                      2,916     3,294           -378    -11.5%\n 48 Calumet Heights              13,812     15,974         -2,162   -13.5%\n 49 Roseland                     

The **dat** object has two entries, one for each page. Each entry is a string. So, when you check the length of the item, it may be surprising that its **length** is only 1. That is because the underlying structure is unknown, it is simply a collection of characters. 

In [5]:
length(dat[[1]])

[1] 1

#### Turning each line in the file into a list

We can carry this out one step at a time, but in order to reach some level of abstraction, we turn it
into a loop. First, we initialize the neighborhood list (**nnlist**) with an empty character [first line below].
Next comes the loop for values of the index **i** going from 1 to 2 (recall that the list has only two elements, one
for each page). Since each element is just one long string, we use the **strsplit** string split command to separate
the long string into a list of one string for each line, by using the return character **\n** as the separator
[line 3 in the code snippet below]. We then extract the first element of the resulting list
using the double bracket notation (this is a side effect of the way lists are organized -- if this seems strange, check the R intro document). We subsequently strip the first four lines from this list (these lines do not contain data -- of course the only way we know this is by carefully checking
the structure of the pdf file).

To streamline the resulting data structure (again, a special characteristic of lists) we turn it into a simple
vector by means of **unlist**. This then allows us to concatenate the result to the current **nnlist** (initially,
just an empty character, after the first step it contains the empty character and the first page, and at the
end it has the empty character, the first and the second page).

In [6]:
nnlist <- ""
for (i in 1:2) {
  ppage <- strsplit(dat[[i]],split="\n")
  nni <- ppage[[1]]
  nni <- nni[-(1:4)]
  nnu <- unlist(nni)
  nnlist <- c(nnlist,nnu)
}
length(nnlist)

[1] 79

The resulting list has 79 elements. Now, we still need to strip the first (empty) element, and the last
element, which is nothing but the totals. We thus extract the elements from **2** to **length - 1**.

In [7]:
nnlist <- nnlist[2:(length(nnlist)-1)]
length(nnlist)

[1] 77

The resulting vector consists of 77 elements that are each a string corresponding to a line in the table.

In [8]:
nnlist[1:3]

[1] " 1  Rogers Park              54,991     63,484        -8,493    -13.4%"
[2] " 2  West Ridge               71,942     73,199        -1,257     -1.7%"
[3] " 3  Uptown                   56,362     63,551        -7,189    -11.3%"

### Extracting the population values

We first initialize a vector of zeros to hold the population values. It is the preferred approach to 
initialize a vector first if one knows its size, rather than having it grow by appending rows or columns.
We use the **vector** command and specify the **mode** to **numeric** and give the **length** as the
length of the list.

In [9]:
nnpop <- vector(mode="numeric",length=length(nnlist))
nnpop

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[39] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[77] 0

We again will use a loop to process each element of the list (each line of the table) one by one.
We use the **substr** command to extract the characters between position 27 and 39 (these values
were determined after taking a careful look at the structure of the table). However, there is still a problem, since
the population values contain commas. We now do two things in one line of code. First, we use **gsub**
to substitute the character **,** by an empty**""**. We turn the result into a numeric value by
means of **as.numeric**. We then assign this number to position **i** of the vector. The resulting
vector **nnpop** contains the population for each of the community areas.

In [10]:
for (i in (1:length(nnlist))) {
     popchar <- substr(nnlist[i],start=27,stop=39)
     popval <- as.numeric(gsub(",","",popchar))
     nnpop[i] <- popval
}
nnpop

[1] 54991 71942 56362 39493 31867 94368 64116 80484 11187 37023 25448 18508
[13] 17931 51542 64124 53359 41932 13426 78743 25010 39262 73595 56323 81432
[25] 98514 18001 20567 54881 35912 79288 35769 29283 21390 13391 18238  5918
[37]  2876 21929 17841 11717 25681 25983 49767 31028 10185 31198  2916 13812
[49] 44619  7325 15109 23042 29651  6482  9426 34513 13393 45368 15612 31977
[61] 44377 18109 39894 23139 33355 55628 35505 30654 32602 41081 48743 20034
[73] 26493 19093 22544 12756 56521

### Creating a data frame

In addition to the vector of the population values, we also need a vector of ID values. Since the community
area indicators are simple sequence numbers, we create such a vector to serve as the ID.

In [11]:
nnid <- (1:length(nnlist))
nnid

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
[51] 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
[76] 76 77

We turn the vectors **nnid** and **nnpop** into a data frame using the **data.frame** command.
Since the variable names assigned automatically are not that informative, we force them to
**NID** and **POP2010** using the **names** command. Also, as we did before, we make sure the ID variable
is an integer (for merging in GeoDa) by means of **as.integer( )**.

In [12]:
neighpop <- data.frame(as.integer(nnid),nnpop)
neighpop

as.integer.nnid.,nnpop
1,54991
2,71942
3,56362
4,39493
5,31867
6,94368
7,64116
8,80484
9,11187
10,37023


In [13]:
names(neighpop) <- c("NID","POP2010")
neighpop

NID,POP2010
1,54991
2,71942
3,56362
4,39493
5,31867
6,94368
7,64116
8,80484
9,11187
10,37023


### Create a csv output file

We write the contents of the data frame to a csv file. As before, we use the **row.names=FALSE** option to
avoid the extraneous first column in the output file.

In [14]:
write.csv(neighpop,"Community_Pop.csv",row.names=FALSE)